# Identifying PII in Student Essays
## Authored by: Pratik Chaudhari, Cody Ledford, Manu Achar
## Project Summary
The Kaggle Competition we are participating in is the [PII Data Detection hosted by The Learning Agency Lab](https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/overview). The goal of this competition is to develop a model that detects sensitive personally identifiable information (PII) in student writing. This is necessary to screen and clean educational data so that when released to the public for analysis and archival, the students' risk are mitigated.


## Python Libraries


In [1]:
# !python -m pip install --upgrade pip

import pandas as pd
import numpy as np
# import spacy as sp
import re
import json
import math
from pathlib import Path
from datasets import Dataset
import os
import torch
import torch.nn as nn
from torch import cuda

import scipy.stats as stats
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.font_manager import FontProperties

# !pip install evaluate
# !pip install seqeval
# import evaluate

from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

!pip install seqeval
from seqeval.metrics import recall_score, precision_score
# from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

2024-04-23 18:41:07.934185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 18:41:07.934297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 18:41:08.084490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3b953ebe93228adb37986abdf91f8e17bb7fa672f32ee70cb5f85a8c0cf198b6
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


## Config

In [2]:
class Config():
    def __init__(self, platform, model_name, pretrained_model_name):
        # platform = 'Kaggle'# 
        if platform == 'kaggle':
            pretrained_model_loc = '/kaggle/input/huggingface-bert/' + pretrained_model_name
            data_path = "/kaggle/input/pii-detection-removal-from-educational-data/train.json"
            test_data_path = "/kaggle/input/pii-detection-removal-from-educational-data/test.json"
            model_path_out = '/kaggle/working/models/' + model_name
            submission_path = '/kaggle/working/submission.csv'

        elif platform == 'local':
            pretrained_model_loc = pretrained_model_name
            model_path = '../models/bert_models/' + model_name +'.bin'
            data_path = "../Datasets/Official/train.json"
            submission_path = '../models/submission/'+ model_name + '-submission.csv'

        self.config = {
            'MAX_LEN': 100,
#             'TRAIN_BATCH_SIZE': 4,
#             'VALID_BATCH_SIZE': 2,
            'EPOCHS': 5,
            'LEARNING_RATE':1e-5,
#             'MAX_GRAD_NORM': 10,
            'device': 'cuda' if cuda.is_available() else 'cpu',
            'data_path': data_path,
            'test_data_path': test_data_path,
            'model_path': model_path_out,
            'pretrained_model': BertForTokenClassification.from_pretrained(pretrained_model_loc, num_labels = 13),
            'tokenizer': BertTokenizerFast.from_pretrained(pretrained_model_loc),
#             'threshold': 0.9,
            'return_entity_level_metrics': True,
            'ignore_subwords': True, # DO NOT CHANGE
            'subm_path': submission_path
        }

In [3]:
platform = 'kaggle'
pretrainend_model_name = 'bert-base-cased'
model_num = 1
model_name = 'model-' + str(model_num) + '-' + pretrainend_model_name

config = Config(platform,model_name, pretrainend_model_name).config

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /kaggle/input/huggingface-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Loading Dataset


In [4]:
df_train = pd.read_json(config['data_path'])
df = df_train
# df_test = pd.read_json(config['test_data_path'])

## Preprocessing

First we scoured the data to find the usuable text with our pretrained model with Regex.

In [5]:
labels = ["O","B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
              "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
              "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
              "I-STREET_ADDRESS","I-URL_PERSONAL"]

labels_to_ids = {k: v for v, k in enumerate(labels)}
ids_to_labels = {v: k for v, k in enumerate(labels)}

In [6]:
pattern_space = re.compile('\xa0|\uf0b7|\u200b')
df_train.loc[:,'full_text'] = df_train.loc[:,'full_text'].replace(pattern_space, ' ')
df_train.loc[:,'tokens'] = df_train.loc[:,'tokens'].apply(lambda line: [tok for tok in line if not re.search(pattern_space,tok)])
# [,.!?-]
df_usable_train = df_train.iloc[df_train[~(df_train.tokens.apply(len) != df_train.labels.apply(len))].index]
1-(len(df_usable_train.document))/len(df.document)

0.06184809754664311

In [7]:
# df_test.loc[:,'full_text'] = df_test.loc[:,'full_text'].replace(pattern_space, ' ')
# df_test.loc[:,'tokens'] = df_test.loc[:,'tokens'].apply(lambda line: [tok for tok in line if not re.search(pattern_space,tok)])
# [,.!?-]
# df_usable_test = df_test.iloc[df_test[~(df_test.tokens.apply(len) != df_test.labels.apply(len))].index]
# 1-(len(df_usable_test.document))/len(df.document)


In [8]:
def make_smaller_inputs(dataframe, type):
    """Splits the entire essays into MAX_LEN size blocks and remaps tokens and labels
    """
    df_out = pd.DataFrame(columns = ['tokens','labels','document','document_location'])
    idx_df = 0
    max_len = config['MAX_LEN']
    
    for _,line in dataframe.iterrows():
        location_counter = 0
        tokens = line.tokens
        if type == 'train':
            labels = line.labels
        document = line.document
        items = range(0,len(tokens),max_len)
        
        for i in items:
            df_out.at[idx_df,'tokens'] = tokens[i:i+max_len]
            if type == 'train':
                df_out.at[idx_df,'labels'] = labels[i:i+max_len]
            df_out.at[idx_df,'document'] = document
            df_out.at[idx_df,'document_location'] = location_counter
            location_counter += 1
            idx_df += 1
        
    return df_out

In [9]:
df_model_input_train = make_smaller_inputs(df_usable_train,'train')
print(len(df_model_input_train.index))
df_model_input_train.head(2)

49199


,tokens,labels,document,document_location
0,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...",7,0
1,"['s, , potential, to, be, released, ., Cf, An...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",7,1


In [10]:
# df_model_input_test = make_smaller_inputs(df_test,'predict')
# print(len(df_model_input_test.index))
# df_model_input_test.head(2)

In [11]:
def compare2dict(list1, list2, pii_threshold):
    return len(set(list1).intersection(set(list2))) > 1

In [12]:
df_trainable = df_model_input_train[df_model_input_train.apply(lambda line: compare2dict(line.labels,list(labels_to_ids.keys()),0), axis = 1)]
df_trainable_input = pd.concat([df_trainable] * 10).sample(frac=1)
df_trainable_input.reset_index(drop = True, inplace=True)
df_trainable_input = df_trainable_input[['tokens','labels']]
print(len(df_trainable_input))
df_trainable_input.head(2)

11560


,tokens,labels
0,"[with, recurring, , themes, ., , Because, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[Reflection, -, Visualization, \n\n, (, Email,...","[O, O, O, O, O, O, O, O, O, O, B-NAME_STUDENT,..."


In [13]:
# df_evaluation_input = df_model_input[['tokens','labels']]
# df_evaluation_input.head(2)

In [14]:
# df_prediction_input = df_model_input_test[['tokens','document','document_location']]
# df_prediction_input.head(2)

## PII Inference V2

### Prepping Trainer

Function for tokenizing datasets

In [15]:
def tokenize(line, tokenizer, type):

    tokens = line.tokens
    

    if config['ignore_subwords']:
        length = config['MAX_LEN']
    else:
        length = math.ceil(config['MAX_LEN'] * 1.2)
        

    encoding = tokenizer(tokens,
                         is_split_into_words= True,
                         return_offsets_mapping= True,
                         padding= 'max_length',
                         max_length= length)
    
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    
    if type == 'train' or type == 'eval':
        word_labels = line.labels
    
        temp_list = [0 for _ in range(length - len(word_labels))]
        labels = [labels_to_ids[label] for label in word_labels] + temp_list
        
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        i = 0
    
        if config['ignore_subwords']:
            #         Ignore subword labels
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] != 0:
                    # overwrite label
                    encoded_labels[idx] = labels[i]
                    i += 1
        else:
            #         Extend subword labels
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0:
                    encoded_labels[idx] = labels[i]
                    i += 1
                    
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return {**item}
        
    if type == 'predict':
        document = line.document
        location = line.document_location
        
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        labels = np.zeros(len(tokens), dtype=int)
        i = 0
    
        if config['ignore_subwords']:
            #         Ignore subword labels
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] != 0:
                    # overwrite label
                    encoded_labels[idx] = labels[i]
                    i += 1
        else:
            #         Extend subword labels
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0:
                    encoded_labels[idx] = labels[i]
                    i += 1
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return {**item, 'document': document, 'location': location}

Create Datasets

In [16]:
train_dataset_temp = df_trainable_input.apply(lambda line: tokenize(line, config['tokenizer'],'train'), axis = 1).to_list()
# train_dataset_temp
# train_dataset = Dataset.from_list(pd.DataFrame(data=train_dataset_temp))

# from_list not implemented in this notebook's version of datasets, janky version of the above
train_dataset = Dataset.from_dict({k: [s[k] for s in train_dataset_temp] for k in  train_dataset_temp[0].keys()})
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels'],
    num_rows: 11560
})

In [17]:
# eval_dataset_temp = df_evaluation_input.apply(lambda line: tokenize(line, config['tokenizer'],'eval'), axis = 1)
# # # eval_dataset = Dataset.from_list(eval_dataset_temp)
# eval_dataset = Dataset.from_dict({k: [s[k] for s in eval_dataset_temp] for k in  eval_dataset_temp[0].keys()})
# eval_dataset

In [18]:
# predict_dataset_temp = df_prediction_input.apply(lambda line: tokenize(line, config['tokenizer'],'predict'), axis = 1)
# # predict_dataset = Dataset.from_list(predict_dataset_temp)
# predict_dataset = Dataset.from_dict({k: [s[k] for s in predict_dataset_temp] for k in  predict_dataset_temp[0].keys()})
# predict_dataset

Initialize parts of trainer

In [19]:
model = config['pretrained_model']
data_collator = DataCollatorForTokenClassification(config['tokenizer'])
# metric = evaluate.load("seqeval")

Function for evaluating prediction. Only available online

In [20]:
# def compute_metrics(p):
#     predictions, labels = p
#     predictions = np.argmax(-1)

#     # Remove ignored index (special tokens)
#     true_predictions = [
#         [ids_to_labels[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     true_labels = [
#         [ids_to_labels[l] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]

#     results = metric.compute(predictions=true_predictions, references=true_labels)
    
#     if config['return_entity_level_metrics']:
#         # Unpack nested dictionaries
#         final_results = {}
#         for key, value in results.items():
#             if isinstance(value, dict):
#                 for n, v in value.items():
#                     final_results[f"{key}_{n}"] = v
#             else:
#                 final_results[key] = value
#         return final_results
#     else:
#         return {
#             "precision": results["overall_precision"],
#             "recall": results["overall_recall"],
#             "f1": results["overall_f1"],
#             "accuracy": results["overall_accuracy"],
#         }

In [21]:


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(-1)

    # Remove ignored index (special tokens)
    true_predictions = [
        [ids_to_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ids_to_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

### Training

Initialize Trainer

In [22]:
training_args = TrainingArguments(
    output_dir= config['model_path'],
#     overwrite_output_dir = True,
    do_train = True,
#     do_predict = True,
#     do_eval = False,
#     per_device_eval_batch_size=1,
    auto_find_batch_size=True,
    report_to="none",
    num_train_epochs = config['EPOCHS'],
    learning_rate = config['LEARNING_RATE'],
    save_strategy = 'no',
    disable_tqdm= False,
    no_cuda = False,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
#     eval_dataset = eval_dataset,
    # predict_dataset = predict_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Start training

In [23]:
# Training
if training_args.do_train:
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.save_model()  # Saves the tokenizer too for easy upload

    metrics["train_samples"] = len(train_dataset)

    trainer.log_metrics("train", metrics)
#     trainer.save_metrics("train", metrics)
    # trainer.save_state()

Step,Training Loss
500,0.118400
1000,0.033000
1500,0.018800
2000,0.011600
2500,0.006900
3000,0.004500
3500,0.003100
4000,0.002100
4500,0.001700
5000,0.001300


***** train metrics *****
  epoch                    =        5.0
  total_flos               =  3349325GF
  train_loss               =     0.0142
  train_runtime            = 0:14:34.09
  train_samples            =      11560
  train_samples_per_second =     66.125
  train_steps_per_second   =      8.266


Reassign model from kaggle output

In [24]:
# trainer.model = BertForTokenClassification.from_pretrained(config['model_path'])
# trainer.model.to(config['device'])

### Evaluation

In [25]:
# # Evaluation
# if training_args.do_eval:
# #     logger.info("*** Evaluate ***")

#     metrics = trainer.evaluate()

#  #   max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
#     metrics["eval_samples"] = len(eval_dataset)

#     trainer.log_metrics("eval", metrics)
#     trainer.save_metrics("eval", metrics)

### Prediction

In [26]:
# # Predict
# if training_args.do_predict:
#     # logger.info("*** Predict ***")

#     predictions, labels, metrics = trainer.predict(predict_dataset, metric_key_prefix="predict")
#     predictions = predictions.argmax(-1)

#     # Remove ignored index (special tokens)
#     true_predictions = [
#         [ids_to_labels[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]

# #     trainer.log_metrics("predict", metrics)
# #     trainer.save_metrics("predict", metrics)

### Build Submission

In [27]:
# results = []
# for preds, location, doc in zip(
#     true_predictions, 
#     predict_dataset["location"], 
#     predict_dataset["document"]
# ):
#     results.extend([(doc, location * config['MAX_LEN'] + idx, label) for idx, label in enumerate(preds) if label != 'O'])

In [28]:
# df_submission = pd.DataFrame(results, columns = ['document','token','label'])
# df_submission['row_id'] = df_submission.index
# df_submission_final = df_submission[['row_id','document','token','label']]
# df_submission_final

In [29]:
# df_submission_final.to_csv(config['subm_path'],index = False)

In [30]:
# df_submission_final.to_csv('submission.csv',index = False)